In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [9]:
from nltk.corpus import stopwords
import nltk
stop_words = nltk.corpus.stopwords.words('english')

In [11]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
import nltk
stop_words = nltk.corpus.stopwords.words('english')
from nltk.stem import PorterStemmer
my_stem = PorterStemmer()
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
dictionary = set(w.lower() for w in nltk.corpus.words.words())
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score

In [13]:
df = pd.read_excel(r"C:\Users\adity\Desktop\Columbia\Spring 2021\DS_PP\Assignment\buzzfeed.xlsx")

In [14]:
### remove articles with no text
df = df[~df.text.isna()]

In [15]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.4, random_state=9)

In [16]:
train.head()

,ID,source,date,title,text,fake,date_time
159,BuzzFeed_Real_71-Webpage.json,http://cnn.it,NaT,Bridgegate: Port Authority officials planned t...,Newark (CNN) Former Port Authority executives ...,0,NaT
7,BuzzFeed_Fake_16-Webpage.json,http://100percentfedup.com,2016-09-19,WHOA! NEW DISTURBING VIDEO Shows HILLARY'S Cam...,"On September 15, Hillary â€œapparentlyâ€ held...",1,2016-09-19 19:39:44
140,BuzzFeed_Real_54-Webpage.json,http://www.ifyouonlynews.com,2016-09-25,Debate Commission STUNS Journalists By Siding ...,\n\n16505 SHARES SHARES FacebookTwitter Google...,0,2016-09-25 10:21:37
154,BuzzFeed_Real_67-Webpage.json,http://abcn.ws,NaT,"Terrorist Attacks Likely to Affect 2016 Race, ...",The recent connected bombings in New York and ...,0,NaT
70,BuzzFeed_Fake_73-Webpage.json,http://rightwingnews.com,2016-09-20,Michelle Obama NOT Leaving The White House - H...,Michelle Obama NOT Leaving The White House â€“...,1,2016-09-20 15:41:33


In [19]:
the_df = pd.DataFrame()
## clean/prepare text
for rev, outcome in zip(train.text.tolist(), train.fake.tolist()):
    
    # only keep words (remove other characters)
    tmp_read = re.sub('[^a-zA-Z]+', ' ', rev).lower()

    #Tokenization and remove stop words
    tmp_read = [word for word in tmp_read.split() if word not in stop_words]

    #dictionary words
    dict_read = [word for word in tmp_read if word in dictionary]
    
    # stemming
    tmp_read_stm = [my_stem.stem(word) for word in tmp_read]
    dict_read_stm = [my_stem.stem(word) for word in dict_read]

    # lemmatization
    tmp_read_lem = [lemmatizer.lemmatize(word) for word in tmp_read]
    dict_read_lem = [lemmatizer.lemmatize(word) for word in dict_read]

    
    # rejoin reviews
    tmp_read = ' '.join(tmp_read)
    tmp_read_stm = ' '.join(tmp_read_stm)
    tmp_read_lem = ' '.join(tmp_read_lem)
    
    dict_read = ' '.join(dict_read)
    dict_read_stm = ' '.join(dict_read_stm)
    dict_read_lem = ' '.join(dict_read_lem)


    # add to new df
    tmp = pd.DataFrame([rev], columns=['original review'])
    tmp['body'] = tmp_read
    tmp['body_stem'] = tmp_read_stm
    tmp['body_lem'] = tmp_read_lem
    tmp['body_dict'] = dict_read
    tmp['body_dict_stem'] = dict_read_stm
    tmp['fake'] = outcome

    the_df = the_df.append(tmp, ignore_index=True)

In [20]:
the_df.head()

,original review,body,body_stem,body_lem,body_dict,body_dict_stem,fake
0,Newark (CNN) Former Port Authority executives ...,newark cnn former port authority executives ne...,newark cnn former port author execut new jerse...,newark cnn former port authority executive new...,former port authority new jersey chris christi...,former port author new jersey chri christi mak...,0
1,"On September 15, Hillary â€œapparentlyâ€ held...",september hillary apparently held rally old st...,septemb hillari appar held ralli old student r...,september hillary apparently held rally old st...,september hillary apparently rally old student...,septemb hillari appar ralli old student recrea...,1
2,\n\n16505 SHARES SHARES FacebookTwitter Google...,shares shares facebooktwitter googlepinterestd...,share share facebooktwitt googlepinterestdiggl...,share share facebooktwitter googlepinterestdig...,former senator daniel patrick said everyone op...,former senat daniel patrick said everyon opini...,0
3,The recent connected bombings in New York and ...,recent connected bombings new york new jersey ...,recent connect bomb new york new jersey stab a...,recent connected bombing new york new jersey s...,recent connected new york new jersey stabbing ...,recent connect new york new jersey stab attack...,0
4,Michelle Obama NOT Leaving The White House â€“...,michelle obama leaving white house hillary cli...,michel obama leav white hous hillari clinton t...,michelle obama leaving white house hillary cli...,michelle leaving white house hillary clinton t...,michel leav white hous hillari clinton terrifi...,1


In [21]:
label = np.array(the_df['fake'])
# choose column to use has main text for models
body_text = np.array(the_df['body_lem'])

In [22]:
# unigram vectorizer
# vectorize
my_vec_tfidf_out = TfidfVectorizer()
my_xform_tfidf_out = my_vec_tfidf_out.fit_transform(body_text)
my_pd = pd.DataFrame(my_xform_tfidf_out.toarray())
my_pd.columns = my_vec_tfidf_out.get_feature_names()
X = my_pd.values

# vectorize test data
test_vec=my_vec_tfidf_out.transform(test.text.tolist())
my_pd = pd.DataFrame(test_vec.toarray())
my_pd.columns = my_vec_tfidf_out.get_feature_names()
X_test = my_pd.values

In [13]:
# # unigram/bigram vectorizer
# my_vec_tfidf_out = TfidfVectorizer(ngram_range = (1,2)) 
# my_xform_tfidf_out = my_vec_tfidf_out.fit_transform(body_text)
# my_pd = pd.DataFrame(my_xform_tfidf_out.toarray())
# my_pd.columns = my_vec_tfidf_out.get_feature_names()
# X = my_pd.values

# test_vec=my_vec_tfidf_out.transform(test.text.tolist())
# my_pd = pd.DataFrame(test_vec.toarray())
# my_pd.columns = my_vec_tfidf_out.get_feature_names()
# X_test = my_pd.values

In [23]:
# are the classes balanced?
balance_check = pd.concat([the_df.fake.value_counts(), 
                the_df.fake.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage'))
print(balance_check)

   counts  percentage
0      59    54.12844
1      50    45.87156


### Modeling 

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, f1_score, precision_recall_curve, plot_precision_recall_curve
import matplotlib.pyplot as plt
import timeit
f1_scorer = make_scorer(f1_score)

In [25]:
# scale data
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_test_scaled = scaler.transform(X_test)

In [26]:
# Logistic Regression
start = timeit.default_timer()
logreg = LogisticRegression(penalty='none',random_state=9).fit(X_scaled, label)
stop = timeit.default_timer()

# # cross-validation for model evaluation
cv_mean = np.mean(cross_val_score(LogisticRegression(penalty='none'), X_scaled, label, cv=5, scoring=f1_scorer))
print("best mean cross-validation score: {:.3f}".format(cv_mean))

y_predictions = logreg.predict(X_test_scaled)
print("test-set score: {:.3f}".format(f1_score(np.array(test.fake), y_predictions)))



# print('model time: ', stop - start) 

best mean cross-validation score: 0.659
test-set score: 0.667


In [27]:
# L1 Penalized Logistic Regression
param_gridlasso = {'C': [0.001 , 0.01 , 0.1 , 1 , 10]} 

# cv = number of folds
gridlasso = GridSearchCV(LogisticRegression(penalty='l1',solver='liblinear',random_state=9), 
                    param_grid=param_gridlasso, cv=5, scoring=f1_scorer)

#use meta model methods to fit score and predict model:
start = timeit.default_timer()
gridlasso.fit(X_scaled, label)
stop = timeit.default_timer()

print("best mean cross-validation score: {:.3f}".format(gridlasso.best_score_))
y_predictions = gridlasso.predict(X_test_scaled)
print("test-set score: {:.3f}".format(f1_score(np.array(test.fake), y_predictions)))

print('model time: ', stop - start) 

best mean cross-validation score: 0.686
test-set score: 0.825
model time:  1.125014399999941


**Gradient Boosting**

In [28]:
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {'n_estimators': [400 , 500,600],
             "max_depth":[3,4,5],
             "learning_rate":[.001,.01,.1]} 

# cv = number of folds
gridgb = GridSearchCV(GradientBoostingClassifier(random_state=9), 
                    param_grid=param_grid, cv=5, scoring=f1_scorer)

#use meta model methods to fit score and predict model:
start = timeit.default_timer()
gridgb.fit(X, label)
stop = timeit.default_timer()

print("best parameters: {}".format(gridgb.best_params_))

print("best mean cross-validation score: {:.3f}".format(gridgb.best_score_))
y_predictions = gridgb.predict(X_test)
print("test-set score: {:.3f}".format(f1_score(np.array(test.fake), y_predictions)))

best parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 400}
best mean cross-validation score: 0.621
test-set score: 0.686


In [29]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(learning_rate=0.01, max_depth=3, n_estimators=500, random_state=9)
model.fit(X, label)
y_predictions = model.predict(X_test)
print("test-set score: {:.3f}".format(f1_score(np.array(test.fake), y_predictions)))
print("test-set score: {:.3f}".format(accuracy_score(np.array(test.fake), y_predictions)))

test-set score: 0.694
test-set score: 0.699


**Random Forest**

In [30]:
from sklearn.ensemble import RandomForestClassifier

param_grid = {'n_estimators': [100 , 300,500,800],
             "max_depth":[1,5,10,15]} 

# cv = number of folds
gridrf = GridSearchCV(RandomForestClassifier(random_state=9), 
                    param_grid=param_grid, cv=5, scoring=f1_scorer)

#use meta model methods to fit score and predict model:
start = timeit.default_timer()
gridrf.fit(X, label)
stop = timeit.default_timer()

print("best parameters: {}".format(gridrf.best_params_))
print("best mean cross-validation score: {:.3f}".format(gridrf.best_score_))

y_predictions = gridrf.predict(X_test)
print("test-set score: {:.3f}".format(f1_score(np.array(test.fake), y_predictions)))

best parameters: {'max_depth': 15, 'n_estimators': 500}
best mean cross-validation score: 0.670
test-set score: 0.625


In [54]:
model = RandomForestClassifier(n_estimators=500, max_depth=15,random_state=9)
model.fit(X, label)
y_predictions = model.predict(X_test)
print("test-set score: {:.3f}".format(f1_score(np.array(test.fake), y_predictions)))
print("test-set score: {:.3f}".format(accuracy_score(np.array(test.fake), y_predictions)))

test-set score: 0.914
test-set score: 0.903


**SVC**

In [59]:
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')

param_grid = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'gamma':['sclae','auto'],
             "C":[1,5,8,10,15]} 

# cv = number of folds
gridsvc = GridSearchCV(SVC(random_state=9), 
                    param_grid=param_grid, cv=5, scoring=f1_scorer)

#use meta model methods to fit score and predict model:
start = timeit.default_timer()
gridsvc.fit(X, label)
stop = timeit.default_timer()

print("best parameters: {}".format(gridsvc.best_params_))
print("best mean cross-validation score: {:.3f}".format(gridsvc.best_score_))

y_predictions = gridsvc.predict(X_test)
print("test-set f1-score: {:.3f}".format(f1_score(np.array(test.fake), y_predictions)))
print("test-set acc score: {:.4f}".format(accuracy_score(np.array(test.fake), y_predictions)))

best parameters: {'C': 5, 'gamma': 'auto', 'kernel': 'linear'}
best mean cross-validation score: 0.827
test-set f1-score: 0.878
test-set acc score: 0.8611


In [62]:
# body_lem -- unigram and bigram
from sklearn.svm import SVC
clf = SVC(kernel='linear', C=5, gamma='auto', random_state=9).fit(X, label)
# clf.score(X_test, np.array(test.fake))
y_predictions = clf.predict(X_test)
print("test-set score: {:.4f}".format(f1_score(np.array(test.fake), y_predictions)))
print("test-set score: {:.4f}".format(accuracy_score(np.array(test.fake), y_predictions)))

test-set score: 0.8780
test-set score: 0.8611


**XGBoost**

In [75]:
from xgboost import XGBClassifier
param = {'max_depth':[1,2, 3], 'eta':[1,2], 'n_estimators':[100]}

# cv = number of folds
gridxgb = GridSearchCV(XGBClassifier(random_state=9), 
                    param_grid=param, cv=5, scoring=f1_scorer)
#use meta model methods to fit score and predict model:
start = timeit.default_timer()
gridxgb.fit(X, label)
stop = timeit.default_timer()

y_predictions = gridxgb.predict(X_test)
print("best parameters: {}".format(gridxgb.best_params_))
print("best mean cross-validation score: {:.3f}".format(gridxgb.best_score_))
print("test-set score: {:.3f}".format(f1_score(np.array(test.fake), y_predictions)))

[12:56:03] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:56:04] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:56:04] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:56:05] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607

[12:56:30] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:56:31] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:56:32] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:56:34] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607

In [74]:
xgboost = XGBClassifier(random_state=9, max_depth=2,eta=1,n_estimators=100)

start = timeit.default_timer()
xgboost.fit(X, label)
stop = timeit.default_timer()

# clf.score(X_test, np.array(test.fake))
y_predictions = xgboost.predict(X_test)
print("test-set score: {:.4f}".format(f1_score(np.array(test.fake), y_predictions)))
print("test-set score: {:.4f}".format(accuracy_score(np.array(test.fake), y_predictions)))

[12:55:53] WARNING: /Users/runner/miniforge3/conda-bld/xgboost_1607604592557/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
test-set score: 0.8052
test-set score: 0.7917
